In [43]:
import requests
import json
import openpyxl
import datetime
import re
import pandas as pd
from IPython.display import Image, HTML
url='https://gql.tokopedia.com/'

# sid 729296 for gudang listrik
def get_base(sid):
    appended_data=[]
    page = 0
    flag = True
    name = []
    img_url=[]
    product_url = []
    product_id = []
    while flag == True:
        page = page + 1
        paramData = [{"operationName": "ShopProducts",
                      "variables": {"sid": str(sid), "page": page, "perPage": 80, "keyword": "", "etalaseId": "etalase",
                                    "sort": 1},
                      "query": "query ShopProducts($sid: String!, $page: Int, $perPage: Int, $keyword: String, $etalaseId: String, $sort: Int) {\n  GetShopProduct(shopID: $sid, filter: {page: $page, perPage: $perPage, fkeyword: $keyword, fmenu: $etalaseId, sort: $sort}) {\n    status\n    errors\n    links {\n      prev\n      next\n      __typename\n    }\n    data {\n      name\n      product_url\n      product_id\n      price {\n        text_idr\n        __typename\n      }\n      primary_image {\n        original\n        thumbnail\n        resize300\n        __typename\n      }\n      flags {\n        isSold\n        isPreorder\n        isWholesale\n        isWishlist\n        __typename\n      }\n      campaign {\n        discounted_percentage\n        original_price_fmt\n        start_date\n        end_date\n        __typename\n      }\n      label {\n        color_hex\n        content\n        __typename\n      }\n      label_groups {\n        position\n        title\n        type\n        __typename\n      }\n      badge {\n        title\n        image_url\n        __typename\n      }\n      stats {\n        reviewCount\n        rating\n        __typename\n      }\n      category {\n        id\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"}]
        r = requests.post(
            url=url,
            data=json.dumps(paramData, indent=2)
        )
        k = r.json()
        for i in k[0]['data']['GetShopProduct']['data']:
            name.append(i['name'])
            img_url.append(i['primary_image']['original'])
            product_url.append(i['product_url'])
            product_id.append(i['product_id'])
        if k[0]['data']['GetShopProduct']['links']['next'] == "":
            flag = False
    dic={'name':name,'img_url':img_url,'product_url':product_url,'product_id':product_id}
    return pd.DataFrame(dic)

In [44]:
def path_to_image_html(path):
    return '<img src="'+ path + '" width="240" >'

In [45]:
pd.set_option('display.max_colwidth', None)
image_cols = ['img_url']  #<- define which columns will be used to convert to html
format_dict = {}
for image_col in image_cols:
    format_dict[image_col] = path_to_image_html

In [46]:
df=get_base(729296)
display(HTML(df.to_html(escape=False ,formatters=format_dict)))

,name,img_url,product_url,product_id
0,Schneider ELCB / RCCB Domae 2P 25A 30mA DOMR01225 1P+N 25 Amp,,https://www.tokopedia.com/gudanglistrik/schneider-elcb-rccb-domae-2p-25a-30ma-domr01225-1p-n-25-amp,1966391013
1,Lampu LED Sensor Gerak 12W Putih Ceiling LED Smart Induction Lamp CDL,,https://www.tokopedia.com/gudanglistrik/lampu-led-sensor-gerak-12w-putih-ceiling-led-smart-induction-lamp-cdl,1931656513
2,LED Flood light 50W Kuning Lampu FLOODLIGHT Sorot 50 watt Outdoor IP65,,https://www.tokopedia.com/gudanglistrik/led-flood-light-50w-kuning-lampu-floodlight-sorot-50-watt-outdoor-ip65,1904337868
3,Panasonic Stop Kontak AC WBSJ1214 MWS 3P Style Series Matte Silver,,https://www.tokopedia.com/gudanglistrik/panasonic-stop-kontak-ac-wbsj1214-mws-3p-style-series-matte-silver,1843418450
4,Schneider Stop Kontak AC E8115_WE Arcato Tanpa Neon Socket 3P 13A,,https://www.tokopedia.com/gudanglistrik/schneider-stop-kontak-ac-e8115-we-arcato-tanpa-neon-socket-3p-13a,1784183152
5,Broco Stop Kontak AC Plano E155 White Socket 3P Indikator Lampu,,https://www.tokopedia.com/gudanglistrik/broco-stop-kontak-ac-plano-e155-white-socket-3p-indikator-lampu,1763301279
6,Panasonic Stop Kontak AC SET WBJ1214 - 3K Plug WEJ9111 Outbow WBJ6010,,https://www.tokopedia.com/gudanglistrik/panasonic-stop-kontak-ac-set-wbj1214-3k-plug-wej9111-outbow-wbj6010,1759464097
7,Panasonic Stop Kontak + Saklar 1G Silver WESJP11212 Style Series MWS,,https://www.tokopedia.com/gudanglistrik/panasonic-stop-kontak-saklar-1g-silver-wesjp11212-style-series-mws,1722821226
8,Panasonic Saklar Engkel Silver WESJ78029 + WESJ5941 MWS Style E Series,,https://www.tokopedia.com/gudanglistrik/panasonic-saklar-engkel-silver-wesj78029-wesj5941-mws-style-e-series,1706159531
9,LEGRAND SET 555109 Mobile Socket + 555129 Plug 5 Kaki 16A 3P+N+E IP44,,https://www.tokopedia.com/gudanglistrik/legrand-set-555109-mobile-socket-555129-plug-5-kaki-16a-3p-n-e-ip44,1703087402
